In [ ]:
import numpy as np
from matrepr import mdisplay, mprint

# Matriz de admitancia calculada a partir do modelo desenhado
#######################
""" #A ideia é entrar com os valores z (pu) e o script calcular o resto, mas por enquanto a entrada é a matriz Y
Z_real = np.array([
    [-10.84j,   3.174j,  1j,      0,       0,       0,       0],
    [  3.174j, -3.174j,  0,       0,       0,       0,       0],
    [   1j,      0,     -6j,     5j,       0,       0,       0],
    [   0,       0,      5j,   -5.266j,  0.095j,  0.076j,  0.095j],
    [   0,       0,      0,     0.095j, -0.095j,    0,       0],
    [   0,       0,      0,     0.076j,    0,    -0.076j,    0],
    [   0,       0,      0,     0.095j,    0,       0,    -5.095j]
], dtype=complex)
"""

Y = np.array([
    [-10.84j,   3.174j,  1j,      0,       0,       0,       0],
    [  3.174j, -3.174j,  0,       0,       0,       0,       0],
    [   1j,      0,     -6j,     5j,       0,       0,       0],
    [   0,       0,      5j,   -5.266j,  0.095j,  0.076j,  0.095j],
    [   0,       0,      0,     0.095j, -0.095j,    0,       0],
    [   0,       0,      0,     0.076j,    0,    -0.076j,    0],
    [   0,       0,      0,     0.095j,    0,       0,    -5.095j]
], dtype=complex)

Z_real = Y**-1
#mdisplay(Z_real)
Z_real = np.nan_to_num(Z_real, copy=False)
#mdisplay(Z_real)

# Cria um layout do sistema Matriz_layout
Matriz_layout = (Y != 0).astype(int)
#mdisplay(Matriz_layout)

# o transformador está entre 3 e 4, logo k=3 e m=4
ykm = -5j
omega = 30;
a = 1;

omega_rad = omega * np.pi / 180;
###############################################################################
#Matriz admitancia com os valores nas barras dos transformadores ajustadas de acordo
Y_ajustada  = Y


#Y_ajustada[k,m]
Y_ajustada[2,2] = (a**2 * np.exp(-1j*omega_rad))*ykm  #Ykk
Y_ajustada[2,3] = (-a * np.exp(-1j*omega_rad))*ykm    #Ykm
Y_ajustada[3,3] = (np.exp(-1j*omega_rad))*ykm         #Ymm
Y_ajustada[3,2] = (-a * np.exp(1j*omega_rad))*ykm     #Ymk

Z = np.linalg.inv(Y)

#mdisplay(Y, 'latex')
#mdisplay(Y_ajustada, 'latex')
#mdisplay(Z, 'latex')

def v_curto(barra,n_barras,debugg = 0):
    Z_n = Z[barra-1,barra-1]
    #corrente de curto na barra n (curto trifásico para o terra), assumindo tensão nominal na barra antes do curto (1 pu)
    Icn = (1)/Z_n
    #print(Icn)
    I_matriz_curto_barra_n = np.zeros((n_barras,1), dtype=complex)
    I_matriz_curto_barra_n[barra-1] = -Icn
    v_delta_n = Z @ I_matriz_curto_barra_n.flatten()  # Direct matrix multiplication

    #mprint(v_delta_n)
    v_iniciais = np.ones((n_barras,1)) #é assumindo que as tensões iniciais nas barras é 1pu
    v_n = v_delta_n + v_iniciais.flatten()
    # Trunca valores muito pequenos para prevenir erros de floating point
    v_n[np.abs(v_n) < 1e-10] = 0
    #mdisplay(v_n)

    print(f"Com o curto na barra {barra}, temos as tensões nas barras como:")
    for i in range(len(v_delta_n)):
        print(f"V_barra{i+1} = {v_n[i]:.4f} p.u.")

    #o calculo da correntes deve ser feito a mão pois depende do layout da rede
    # I=V/Z

    I_sistema = np.zeros((n_barras,n_barras), dtype=complex)
    for i in range(n_barras):
      for j in range(n_barras):
        if i != j and Matriz_layout[i, j] != 0: # Only calculate current if connected
          I_sistema[i,j] = (v_n[i] - v_n[j])/Z_real[i,j]
    I_sistema[np.abs(I_sistema) < 1e-10] = 0
    I_fluxo = I_sistema * Matriz_layout


    print("\nFluxo de Corrente:")
    for i in range(n_barras):
        for j in range(n_barras):
            if abs(I_fluxo[i, j]) > 1e-10:  # Check for non-zero current flow
                print(f"A corrente {I_fluxo[i, j]:.4f} p.u. flui da barra {i+1} para a barra {j+1}")

#######################################################################################################
#conferir as conexões determinadas pela matriz de layout
    if debugg:
      print("\nConexoes:")
      for i in range(n_barras):
          for j in range(n_barras):
            if abs(Matriz_layout[i, j]) > 0:  # Check for non-zero current flow
              print(f"a barra {i+1} se conecta a barra {j+1}")
      print("\nmatriz de layout")
      mdisplay(Matriz_layout)
      print("\n matriz Y crua")
      mdisplay(Y)
      print("\n matriz Y considerando o transformador")
      mdisplay(Y_ajustada)
      print("\n matriz Z calculada a partir de Y , Z=Y^-1")
      mdisplay(Z)
      print(f"\n Z da barra {barra} em curto para o terra é de Z_n {Z_n} e a currente de curto é de {Icn}")
      print("\n Z real obtida do sistema")
      mdisplay(Z_real)
      print("\n v_delta_n")
      mprint(v_delta_n)
      print("\n v_n")
      mprint(v_n)
      print("Correntes no sistema:")
      mdisplay(I_sistema)
      print("Correntes no fluxo:")
      mdisplay(I_fluxo)





    #return v_n

v_curto(1,7)

C:\Users\jooj\AppData\Local\Temp\ipykernel_17716\745715757.py:28: RuntimeWarning: invalid value encountered in reciprocal
  Z_real = Y**-1


,0,1,2,3,4,5,6
0,-0+0.09225i,0--0.3151i,0--1i,nan+nani,nan+nani,nan+nani,nan+nani
1,0--0.3151i,-0+0.3151i,nan+nani,nan+nani,nan+nani,nan+nani,nan+nani
2,0--1i,nan+nani,-0+0.1667i,0--0.2i,nan+nani,nan+nani,nan+nani
3,nan+nani,nan+nani,0--0.2i,-0+0.1899i,0--10.53i,0--13.16i,0--10.53i
4,nan+nani,nan+nani,nan+nani,0--10.53i,-0+10.53i,nan+nani,nan+nani
5,nan+nani,nan+nani,nan+nani,0--13.16i,nan+nani,-0+13.16i,nan+nani
6,nan+nani,nan+nani,nan+nani,0--10.53i,nan+nani,nan+nani,-0+0.1963i


,0,1,2,3,4,5,6
0,-0+0.09225i,0--0.3151i,0--1i,0+0i,0+0i,0+0i,0+0i
1,0--0.3151i,-0+0.3151i,0+0i,0+0i,0+0i,0+0i,0+0i
2,0--1i,0+0i,-0+0.1667i,0--0.2i,0+0i,0+0i,0+0i
3,0+0i,0+0i,0--0.2i,-0+0.1899i,0--10.53i,0--13.16i,0--10.53i
4,0+0i,0+0i,0+0i,0--10.53i,-0+10.53i,0+0i,0+0i
5,0+0i,0+0i,0+0i,0--13.16i,0+0i,-0+13.16i,0+0i
6,0+0i,0+0i,0+0i,0--10.53i,0+0i,0+0i,-0+0.1963i


Com o curto na barra 1, temos as tensões nas barras como:
V_barra1 = 0.0000+0.0000j p.u.
V_barra2 = 0.0000+0.0000j p.u.
V_barra3 = 1.0032-0.1939j p.u.
V_barra4 = 1.1764-0.0939j p.u.
V_barra5 = 1.1764-0.0939j p.u.
V_barra6 = 1.1764-0.0939j p.u.
V_barra7 = 1.0033-0.0018j p.u.

Fluxo de Corrente:
A corrente -0.1939-1.0032j p.u. flui da barra 1 para a barra 3
A corrente 0.1939+1.0032j p.u. flui da barra 3 para a barra 1
A corrente 0.5000-0.8660j p.u. flui da barra 3 para a barra 4
A corrente -0.5000+0.8660j p.u. flui da barra 4 para a barra 3
A corrente 0.0088+0.0165j p.u. flui da barra 4 para a barra 7
A corrente -0.0088-0.0165j p.u. flui da barra 7 para a barra 4
